In [47]:
using JuMP, HiGHS, LinearAlgebra

In [48]:
# Maze Specific Values for small maze


m_small = 3
n_small = 3
start_row_small= 3
start_col_small= 1
end_row_small= 3
end_col_small= 2

x_small = [0 1 1 0
     0 1 0 0
     0 0 1 0]

y_small = [0 0 0 
     1 1 1 
     1 0 1
     0 0 0]

4×3 Matrix{Int64}:
 0  0  0
 1  1  1
 1  0  1
 0  0  0

In [49]:
maze_small = Model(HiGHS.Optimizer)
@variable(maze_small, a[0:(m_small+1),0:(n_small+1)], Bin)

@constraint(maze_small, [j=0:(n_small+1)], a[0,j] == 0)
@constraint(maze_small, [j=0:(n_small+1)], a[m_small+1,j] == 0)
@constraint(maze_small, [i=0:(m_small+1)], a[i,0] == 0)
@constraint(maze_small, [i=0:(m_small+1)], a[i,n_small+1] == 0)
@constraint(maze_small, a[start_row_small,start_col_small]==1)
@constraint(maze_small, a[end_row_small,end_col_small]==1)

# A node on the path must have EXACTLY TWO neighbors on the path which are also accessible,
#   but nodes not on the path can have 0, 1, or 2 accessible nodes on the path.
@constraint(maze_small, [i=1:m_small, j=1:n_small; !((i==start_row_small && j==start_col_small) || (i==end_row_small && j==end_col_small))], 
    x_small[i,j]*a[i,j-1] + x_small[i,j+1]*a[i,j+1] + y_small[i,j]*a[i-1,j] + y_small[i+1,j]*a[i+1,j] - 2*a[i,j] >= 0)

# A node NOT on the path has NO MORE THAN TWO accessible neighbors on the path
#   0 = blocked from or not adjacent to minimial path
#   1 = accessible and adjacent to minimal path, but minimal path goes another way
#   2 = minimal path runs a corner around a node not on path accessible on two sides, but 
#           it would not shorten the path to take that node
@constraint(maze_small, [i=1:m_small, j=1:n_small; !((i==start_row_small && j==start_col_small) || (i==end_row_small && j==end_col_small))], 
    x_small[i,j]*a[i,j-1] + x_small[i,j+1]*a[i,j+1] + y_small[i,j]*a[i-1,j] + y_small[i+1,j]*a[i+1,j] <= 2)

# Start node has EXACTLY one accessible neighbor on the path 
@constraint(maze_small, x_small[start_row_small,start_col_small]*a[start_row_small,(start_col_small-1)] + x_small[start_row_small,(start_col_small+1)]*a[start_row_small,(start_col_small+1)] 
    + y_small[start_row_small,start_col_small]*a[(start_row_small-1),start_col_small] + y_small[(start_row_small+1),start_col_small]*a[(start_row_small+1),start_col_small] == 1)

# End node has EXACTLY 1 accessible neighbor on the path
@constraint(maze_small, x_small[end_row_small,end_col_small]*a[end_row_small,(end_col_small-1)] + x_small[end_row_small,(end_col_small+1)]*a[end_row_small,(end_col_small+1)]
    + y_small[end_row_small,end_col_small]*a[(end_row_small-1),end_col_small] + y_small[(end_row_small+1),end_col_small]*a[(end_row_small+1),end_col_small] == 1)

o = @objective(maze_small, Min, sum(a[1:m_small,1:n_small]))
print(maze_small)

In [50]:
set_silent(maze_small)
optimize!(maze_small)
is_solved_and_feasible(maze_small)

true

In [51]:
objective_value(maze_small)

8.0

In [52]:
round.(Int, value.(a[1:m_small, 1:n_small]))

2-dimensional DenseAxisArray{Int64,2,...} with index sets:
    Dimension 1, [1, 2, 3]
    Dimension 2, [1, 2, 3]
And data, a 3×3 Matrix{Int64}:
 1  1  1
 1  0  1
 1  1  1

In [53]:
include("SimplexTableau.jl")
using .SimplexTableau

Our problem begins:
$$\begin{aligned}
\text{Minimize} && w = \sum_{i=1}^3 \sum_{j=1}^3 a_{ij},& \\
\text{subject to} && a_{31} = 1, \\
&& a_{32} = 1,\\
&& a_{21} + 0a_{32} = 1, \\
&& 0a_{22} + a_{33} = 1, \\
&& x_{ij} \cdot a_{i(j-1)} + x_{i(j+1)} \cdot a_{i(j+1)} + y_{ij} \cdot a_{(i-1)j} + y_{(i+1)j} \cdot a_{(i+1)j} - 2 \cdot a_{ij} \geq 0, \\
&& x_{ij} \cdot a_{i(j-1)} + x_{i(j+1)} \cdot a_{i(j+1)} + y_{ij} \cdot a_{(i-1)j} + y_{(i+1)j} \cdot a_{(i+1)j} \leq 2 \\
\text{and}&& a_{ij} \in \{0,1\} \\
\text{for}&& 1 \leq i \leq 3  \text{ and } 1 \leq j \leq 3.
\end{aligned}
$$

Which we can rewrite as:
$$\begin{aligned}
\text{Maximize} && z = -w = -\sum_{i=1}^3 \sum_{j=1}^3 a_{ij},& \\
\text{subject to} && a_{31} = 1, \\
&& a_{32} = 1,\\
&& a_{21} + 0a_{32} = 1, \\
&& 0a_{22} + a_{33} = 1, \\
&& 2 \cdot a_{ij} - x_{ij} \cdot a_{i(j-1)} - x_{i(j+1)} \cdot a_{i(j+1)} - y_{ij} \cdot a_{(i-1)j} - y_{(i+1)j} \cdot a_{(i+1)j} \leq 0, \\
&& x_{ij} \cdot a_{i(j-1)} + x_{i(j+1)} \cdot a_{i(j+1)} + y_{ij} \cdot a_{(i-1)j} + y_{(i+1)j} \cdot a_{(i+1)j} \leq 2 \\
\text{and}&& a_{ij} \in \{0,1\} \\
\text{for}&& 1 \leq i \leq 3  \text{ and } 1 \leq j \leq 3.
\end{aligned}
$$

In [54]:
A = [0 0 0 0 0 0 1 0 0
     0 0 0 0 0 0 0 1 0
     0 0 0 1 0 0 0 0 0
     0 0 0 0 0 0 0 0 1
     2 -1 0 -1 0 0 0 0 0
     -1 2 -1 0 0 -1 0 0 0
     0 -1 2 0 0 -1 0 0 0
     -1 0 0 2 -1 0 -1 0 0
     0 -1 0 -1 2 0 0 0 0
     0 0 -1 0 0 2 0 0 -1
     0 0 0 0 0 -1 0 -1 2
     0 1 0 1 0 0 0 0 0
     1 0 1 0 1 0 0 0 0
     0 1 0 0 0 1 0 0 0
     1 0 0 0 1 0 1 0 0 
     0 0 1 0 0 0 0 0 1
     0 0 0 0 0 1 0 1 0]
b = [1; 1; 1; 1; 0; 0; 0; 0; 0; 0; 0; 2; 2; 2; 2; 2; 2]
c = [-1; -1; -1; -1; -1; -1; -1; -1; -1]
t = Tableau(A, b, c,equality=[1;2;3;4])

LoadError: UndefVarError: `Tableau` not defined in `Main`
Hint: It looks like two or more modules export different bindings with this name, resulting in ambiguity. Try explicitly importing it from a particular module, or qualifying the name with the module it should come from.

Phase 0:

In [ ]:
t1 = phase0(t)

A =
 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 2 -1  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  1
-1  2 -1  0  0 -1  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 0 -1  2  0  0 -1  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
-1  0  0  0 -1  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 -1
 0 -1  0  0  2  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  1
 0  0 -1  0  0  2  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  1
 0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0 -1
 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1
 1  0  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  2
 0  1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0 

Phase 1:

In [ ]:
t2 = pivot(t1, [1 => 13, 2 => 10, 3 => 11, 5 => 12, 6 => 16, 10 => 17, 12 => 15, 17 => 12])

A =
 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  1  0  0  0  0  0  0  0  0  0 -1  0  0 -2 -3  0  0  0  0  0  0  0  1
 0  0  1  0  0  0  0  0  0  0  0  0  0  0 -1 -2  0  0  0  0  0  0  0  1
 0  0  0  0  1  0  0  0  0  0  1  2 -1  0  3  3  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0 -1 -2  0  0 -3 -3  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0 -2 -5  2  1 -8 -9  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  2  3  1  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  1  2  3  0  0  4  3  0  0  0  0  0  0  0  2
 0  0  0  0  0  0  0  0  0  0  0  0  1  0  1  2  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  2  4  0  0  1  0 

In [ ]:
primal_optimal = t2

A =
 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  1  0  0  0  0  0  0  0  0  0 -1  0  0 -2 -3  0  0  0  0  0  0  0  1
 0  0  1  0  0  0  0  0  0  0  0  0  0  0 -1 -2  0  0  0  0  0  0  0  1
 0  0  0  0  1  0  0  0  0  0  1  2 -1  0  3  3  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0 -1 -2  0  0 -3 -3  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0 -2 -5  2  1 -8 -9  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  2  3  1  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  1  2  3  0  0  4  3  0  0  0  0  0  0  0  2
 0  0  0  0  0  0  0  0  0  0  0  0  1  0  1  2  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  2  4  0  0  1  0 

$\vec{x} = (0, 1, 1, 1, 1, 1, 1, 1, 1 \ | \ 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0); \qquad \qquad w = -z = 8$

$\vec{x}'$ is of form $(0, +, +, +, +, +, +, +, + \ | \ +, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)$, so 

$\vec{y}'$ should be of form $( 0, *, *, *, *, *, *, *, *, *, *, *, * \ | \ *, 0, 0, 0, 0, 0, 0, 0, 0)$

but $\vec{y}'$ is of form $( 0, 0, *, *, 0, *, *, 0, 0, 0, 0, 0, 0 \ | \ 0, 0, 0, 0, 0, 0, 0, 0, 0)$.

In [56]:
primal_2nd_solution = pivot(primal_optimal, [11 => 10])

LoadError: UndefVarError: `pivot` not defined in `Main`
Hint: It looks like two or more modules export different bindings with this name, resulting in ambiguity. Try explicitly importing it from a particular module, or qualifying the name with the module it should come from.

$\vec{x} = (1, 1, 1, 1, 0, 1, 1, 1, 1 \ | \ 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0); \qquad \qquad w = -z = 8$

$\vec{x}'$ is of form $(+, +, +, +, 0, +, +, +, + \ | \ 0, +, 0, 0, 0, +, 0, 0, 0, 0, 0, 0, 0)$, so 

$\vec{y}'$ is of form $( *, 0, *, *, 0, *, *, *, *, *, *, *, * \ | \ 0, 0, 0, 0, *, 0, 0, 0, 0)$.

Dual Problem:

$$\begin{aligned}
\text{Minimize} && w = \vec{b}^T\vec{y},& \\
\text{subject to} && A^T\vec{y} \geq \vec{c}, \\
\text{and} && y_k \geq 0  \text{ for } 1 \leq k \leq 17\\
\end{aligned}
$$

Which becomes: 
$$\begin{aligned}
\text{Maximize} && z = -w = -\vec{b}^T\vec{y},& \\
\text{subject to} && -A^T\vec{y} \leq -\vec{c}, \\
\text{and} && y_k \geq 0  \text{ for } 1 \leq k \leq 17\\
\end{aligned}
$$

In [ ]:
A_no_eq = [2 -1 0 -1 0 0 0 0 0
          -1 2 -1 0 0 -1 0 0 0
           0 -1 2 0 0 -1 0 0 0
          -1 0 0 2 -1 0 -1 0 0
           0 -1 0 -1 2 0 0 0 0
           0 0 -1 0 0 2 0 0 -1
           0 0 0 0 0 -1 0 -1 2
           0 1 0 1 0 0 0 0 0
           1 0 1 0 1 0 0 0 0
           0 1 0 0 0 1 0 0 0
           1 0 0 0 1 0 1 0 0 
           0 0 1 0 0 0 0 0 1
           0 0 0 0 0 1 0 1 0]
           b_no_eq = [0; 0; 0; 0; 0; 0; 0; 2; 2; 2; 2; 2; 2]
dual = Tableau(-1*Transpose(A_no_eq), c, -b_no_eq)

A =
-2  1  0  1  0  0  0  0 -1  0 -1  0  0  1  0  0  0  0  0  0  0  0  0 -1
 1 -2  1  0  1  0  0 -1  0 -1  0  0  0  0  1  0  0  0  0  0  0  0  0 -1
 0  1 -2  0  0  1  0  0 -1  0  0 -1  0  0  0  1  0  0  0  0  0  0  0 -1
 1  0  0 -2  1  0  0 -1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1
 0  0  0  1 -2  0  0  0 -1  0 -1  0  0  0  0  0  0  1  0  0  0  0  0 -1
 0  1  1  0  0 -2  1  0  0 -1  0  0 -1  0  0  0  0  0  1  0  0  0  0 -1
 0  0  0  1  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  1  0  0  0 -1
 0  0  0  0  0  0  1  0  0  0  0  0 -1  0  0  0  0  0  0  0  1  0  0 -1
 0  0  0  0  0  1 -2  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  1  0 -1
 0  0  0  0  0  0  0  2  2  2  2  2  2  0  0  0  0  0  0  0  0  0  1  0

d = 1
π = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}
β = {14, 15, 16, 17, 18, 19, 20, 21, 22}


In [ ]:
dual1 = phaseI(dual, show_steps = true)

SimplexSteps(A =
-2  1  0  1  0  0  0  0 -1  0 -1  0  0  1  0  0  0  0  0  0  0  0  0 -1
 1 -2  1  0  1  0  0 -1  0 -1  0  0  0  0  1  0  0  0  0  0  0  0  0 -1
 0  1 -2  0  0  1  0  0 -1  0  0 -1  0  0  0  1  0  0  0  0  0  0  0 -1
 1  0  0 -2  1  0  0 -1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1
 0  0  0  1 -2  0  0  0 -1  0 -1  0  0  0  0  0  0  1  0  0  0  0  0 -1
 0  1  1  0  0 -2  1  0  0 -1  0  0 -1  0  0  0  0  0  1  0  0  0  0 -1
 0  0  0  1  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  1  0  0  0 -1
 0  0  0  0  0  0  1  0  0  0  0  0 -1  0  0  0  0  0  0  0  1  0  0 -1
 0  0  0  0  0  1 -2  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  1  0 -1
 0  0  0  0  0  0  0  2  2  2  2  2  2  0  0  0  0  0  0  0  0  0  1  0

d = 1
π = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}
β = {14, 15, 16, 17, 18, 19, 20, 21, 22}
, A =
  32    0    0    0    0    0  -16   16   40   16    0    0    0  -24  -16   -8    0   -8    0   32    0    8    0   16
   0    0    0    0    0   32  -40   24  

Phase 2:

In [ ]:
dual2 = phaseII(dual1.final,show_steps = true)

SimplexSteps(A =
  32    0    0    0    0    0  -16   16   40   16    0    0    0  -24  -16   -8    0   -8    0   32    0    8    0   16
   0    0    0    0    0   32  -40   24   44   40    0    0    0  -12  -24  -20    0  -12  -16   24   16   20    0   24
   0    0    0    0    0    0  -32    0    0    0    0    0   32    0    0    0    0    0    0    0  -32    0    0   32
   0    0    0    0    0    0   -8   24   28    8   32    0    0  -12   -8   -4  -16  -12    0   -8    0    4    0   56
   0   32    0    0    0    0  -32   32   48   32    0    0    0  -16  -32  -16    0  -16    0   32    0   16    0   32
   0    0    0    0    0    0   24   24   44   40    0   32    0  -12  -24  -20    0  -12  -16   24   16  -12    0   56
   0    0    0    0   32    0    0    0   16    0    0    0    0    0    0    0    0  -16    0   16    0    0    0    0
   0    0    0   32    0    0   -8   24   28    8    0    0    0  -12   -8   -4  -16  -12    0   24    0    4    0   24
   0    0   32    0    

$\vec{y} = (0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1 \ | \ 0, 0, 0, 0, 0, 0, 0, 0, 0) \qquad \qquad w = -z = 8$

$\vec{y}'$ should be of the form: $( 0, *, *, *, *, *, *, *, *, *, *, *, * \ | \ *, 0, 0, 0, 0, 0, 0, 0, 0)$, 

which $y$ is. And the primal objective matches the dual objective, meaning that our objective $w=8$ is optimal.

In [ ]:
y = [0; 0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 1; 1]
dual_solution = transpose(b_no_eq)*y

8

Medium Size Maze:

In [ ]:
# Maze Specific Values


m_med = 10
n_med = 10
start_row_med= 2
start_col_med= 2
end_row_med= 10
end_col_med= 10

x_med = [0 1 1 1 0 0 0 0 1 1 0
     0 0 0 1 0 0 1 1 0 0 0 
     0 0 1 1 0 0 0 0 1 0 0
     0 0 1 1 0 0 0 0 1 1 0
     0 1 1 1 0 1 0 1 0 0 0
     0 1 1 0 1 0 1 0 0 0 0 
     0 1 1 1 0 1 0 1 0 0 0
     0 1 1 1 1 0 1 1 1 1 0
     0 1 1 1 0 1 1 1 1 0 0
     0 1 0 1 1 0 1 1 1 1 0]  

y_med = [0 0 0 0 0 0 0 0 0 0 
     1 1 0 1 1 1 1 1 0 1
     1 0 1 0 1 1 1 0 1 1 
     1 1 0 1 1 1 0 1 1 1
     0 0 1 0 1 1 1 1 0 1
     0 0 1 0 1 0 1 1 1 1
     1 1 0 1 0 1 0 0 1 1 
     0 0 0 0 1 0 1 1 1 0
     1 0 0 1 0 1 1 0 0 1
     1 0 1 0 1 1 0 0 1 0
     0 0 0 0 0 0 0 0 0 0]

11×10 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0
 1  1  0  1  1  1  1  1  0  1
 1  0  1  0  1  1  1  0  1  1
 1  1  0  1  1  1  0  1  1  1
 0  0  1  0  1  1  1  1  0  1
 0  0  1  0  1  0  1  1  1  1
 1  1  0  1  0  1  0  0  1  1
 0  0  0  0  1  0  1  1  1  0
 1  0  0  1  0  1  1  0  0  1
 1  0  1  0  1  1  0  0  1  0
 0  0  0  0  0  0  0  0  0  0

In [ ]:
maze_med = Model(HiGHS.Optimizer)
@variable(maze_med, a[0:(m_med+1),0:(n_med+1)], Bin)

@constraint(maze_med, [j=0:(n_med+1)], a[0,j] == 0)
@constraint(maze_med, [j=0:(n_med+1)], a[m_med+1,j] == 0)
@constraint(maze_med, [i=0:(m_med+1)], a[i,0] == 0)
@constraint(maze_med, [i=0:(m_med+1)], a[i,n_med+1] == 0)
@constraint(maze_med, a[start_row_med,start_col_med]==1)
@constraint(maze_med, a[end_row_med,end_col_med]==1)

# A node on the path must have EXACTLY TWO neighbors on the path which are also accessible,
#   but nodes not on the path can have 0, 1, or 2 accessible nodes on the path.
@constraint(maze_med, [i=1:m_med, j=1:n_med; !((i==start_row_med && j==start_col_med) || (i==end_row_med && j==end_col_med))], 
    x_med[i,j]*a[i,j-1] + x_med[i,j+1]*a[i,j+1] + y_med[i,j]*a[i-1,j] + y_med[i+1,j]*a[i+1,j] - 2*a[i,j] >= 0)

# A node NOT on the path has NO MORE THAN TWO accessible neighbors on the path
#   0 = blocked from or not adjacent to minimial path
#   1 = accessible and adjacent to minimal path, but minimal path goes another way
#   2 = minimal path runs a corner around a node not on path accessible on two sides, but 
#           it would not shorten the path to take that node
@constraint(maze_med, [i=1:m_med, j=1:n_med; !((i==start_row_med && j==start_col_med) || (i==end_row_med && j==end_col_med))], 
    x_med[i,j]*a[i,j-1] + x_med[i,j+1]*a[i,j+1] + y_med[i,j]*a[i-1,j] + y_med[i+1,j]*a[i+1,j] <= 2)

# Start node has EXACTLY one accessible neighbor on the path 
@constraint(maze_med, x_med[start_row_med,start_col_med]*a[start_row_med,(start_col_med-1)] + x_med[start_row_med,(start_col_med+1)]*a[start_row_med,(start_col_med+1)] 
    + y_med[start_row_med,start_col_med]*a[(start_row_med-1),start_col_med] + y_med[(start_row_med+1),start_col_med]*a[(start_row_med+1),start_col_med] == 1)

# End node has EXACTLY 1 accessible neighbor on the path
@constraint(maze_med, x_med[end_row_med,end_col_med]*a[end_row_med,(end_col_med-1)] + x_med[end_row_med,(end_col_med+1)]*a[end_row_med,(end_col_med+1)]
    + y_med[end_row_med,end_col_med]*a[(end_row_med-1),end_col_med] + y_med[(end_row_med+1),end_col_med]*a[(end_row_med+1),end_col_med] == 1)

    random_med = rand(0.99:0.001:1.01, m_med, n_med)

o = @objective(maze_med, Min, sum(random_med[i,j]*a[i,j] for i in 1:m_med, j in 1:n_med))

set_silent(maze_med)
optimize!(maze_med)
is_solved_and_feasible(maze_med)
objective_value(maze_med)
round.(Int, value.(a[1:m_med, 1:n_med]))

2-dimensional DenseAxisArray{Int64,2,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    Dimension 2, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
And data, a 10×10 Matrix{Int64}:
 0  1  1  1  0  0  0  1  1  1
 0  1  1  1  0  1  1  1  0  1
 0  0  1  1  0  1  0  0  0  1
 0  0  1  1  0  1  0  1  1  1
 0  0  1  0  1  1  1  1  0  0
 0  1  1  1  1  1  1  0  0  0
 0  1  1  1  1  1  0  0  0  0
 0  0  0  1  1  0  0  0  0  0
 0  0  1  1  1  1  0  0  0  0
 0  0  1  1  1  1  1  1  1  1



Large Maze: 

In [ ]:
m = 50
n = 100
start_row = 25
start_col = 1
end_row = 25
end_col = 100

x = [0  0
     0  0
     0  0
     0  0
     0  0
     0  0
     0  0
     0  0
     0  0
     0  0
     0  0
     0  0
     0  0
     0  0
     0  0
     0  0
     0  0
     0  0
     0  0
     0  0]

y = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

     0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


2×30 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

In [ ]:
maze2 = Model(HiGHS.Optimizer)
@variable(maze2, a[0:(m+1),0:(n+1)], Bin)

@constraint(maze2, [j=0:(n+1)], a[0,j] == 0)
@constraint(maze2, [j=0:(n+1)], a[m+1,j] == 0)
@constraint(maze2, [i=0:(m+1)], a[i,0] == 0)
@constraint(maze2, [i=0:(m+1)], a[i,n+1] == 0)
@constraint(maze2, a[start_row,start_col]==1)
@constraint(maze2, a[end_row,end_col]==1)

# A node on the path must have EXACTLY TWO neighbors on the path which are also accessible,
#   but nodes not on the path can have 0, 1, or 2 accessible nodes on the path.
@constraint(maze2, [i=1:m, j=1:n; !((i==start_row && j==start_col) || (i==end_row && j==end_col))], 
    x[i,j]*a[i,j-1] + x[i,j+1]*a[i,j+1] + y[i,j]*a[i-1,j] + y[i+1,j]*a[i+1,j] - 2*a[i,j] >= 0)

# A node NOT on the path has NO MORE THAN TWO accessible neighbors on the path
#   0 = blocked from or not adjacent to minimial path
#   1 = accessible and adjacent to minimal path, but minimal path goes another way
#   2 = minimal path runs a corner around a node not on path accessible on two sides, but 
#           it would not shorten the path to take that node
@constraint(maze2, [i=1:m, j=1:n; !((i==start_row && j==start_col) || (i==end_row && j==end_col))], 
    x[i,j]*a[i,j-1] + x[i,j+1]*a[i,j+1] + y[i,j]*a[i-1,j] + y[i+1,j]*a[i+1,j] <= 2)

# Start node has EXACTLY one accessible neighbor on the path 
@constraint(maze2, x[start_row,start_col]*a[start_row,(start_col-1)] + x[start_row,(start_col+1)]*a[start_row,(start_col+1)] 
    + y[start_row,start_col]*a[(start_row-1),start_col] + y[(start_row+1),start_col]*a[(start_row+1),start_col] == 1)

# End node has EXACTLY 1 accessible neighbor on the path
@constraint(maze2, x[end_row,end_col]*a[end_row,(end_col-1)] + x[end_row,(end_col+1)]*a[end_row,(end_col+1)]
    + y[end_row,end_col]*a[(end_row-1),end_col] + y[(end_row+1),end_col]*a[(end_row+1),end_col] == 1)


o = @objective(maze2, Min, sum(a[1:m,1:n]))


LoadError: BoundsError: attempt to access 20×2 Matrix{Int64} at index [1, 3]

In [ ]:
set_silent(maze2)
optimize!(maze2)
is_solved_and_feasible(maze2)

: 

: 

In [ ]:
objective_value(maze2)

3476.386999999998

In [ ]:
round.(Int, value.(a[1:m, 1:n]))

2-dimensional DenseAxisArray{Int64,2,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
    Dimension 2, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
And data, a 50×100 Matrix{Int64}:
 0  0  0  0  0  1  1  1  0  0  0  0  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  1  1  1  0  1  1  1  0  0  0  1  1     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  1  1  0  0  0  0  0  0  1  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  1  0  0  1  1  1  1  0  1  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  1  1  0  0  1  0  0  1  0  1  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  1  1  1  1  1  0  0  1  1  1  0  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  1  0  0  0  0  0  0  0  0  1  1  1     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  1  0  0  1  1  0  1  1  1  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  1  1  0  1  1  1  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  1  0  0  0  0